In [1]:
from selenium.webdriver import Chrome, ChromeOptions
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import  WebDriverWait
import pandas as pd 
import time, threading, os, requests, ast,re
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait

In [2]:
def scarpe(auction_id):
    path = f"https://www.centralcarauctions.com/auction/{auction_id}"
    options = ChromeOptions()
    options.add_argument("--start-maximized")
    service = Service(ChromeDriverManager().install())
    driver = Chrome(service=service, options=options)
    driver.get(path)
    driver.maximize_window()
    
    
    try:
        cookie_btn = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.ID, "ccc-notify-accept"))
        )
        driver.execute_script("arguments[0].scrollIntoView(true);", cookie_btn)
        cookie_btn.click()
        print("🍪 Cookie popup accepted")
        time.sleep(1)
    except Exception:
        print("No cookie popup found")
    try:

        login = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, './/a[text() = "Login"]')))
        if login:
            login.click()
    except Exception as e:
        print("no login")
    try:
        provided_u_name = "fourbrotherstrading@icloud.com"
        user_name = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.ID,'username')))   
        user_name.send_keys(provided_u_name)
    except Exception as e:
        print(f"No username tab found and the error is {e}")
    
    try:
        provided_pass = "Sultanmirza1501#"
        password = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.ID, 'password')))
        driver.execute_script("arguments[0].scrollIntoView();", password)
        password.send_keys(provided_pass)
    except Exception as e:
        print(f"No password tab found and the error is {e}") 
    
    try:
        check = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.XPATH, './/button[text() = "Sign in"]')))
        if check:
            check.click()
            time.sleep(2)
            for i in range(2):
                driver.back()
    except Exception as e:
        print(f"No check tab found and the error is {e}") 
        
    try:
        cars = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, './/div[@class="pagination-pages"]/span[last()]'))).text.strip("()")
        if cars:
            splited_cars = cars.split(" ")
            total_cars = int(splited_cars[0])
            print(f"{total_cars} number of cars found")
        else:
            print("No cars found")
    except Exception as e:
        print(f"No cars found and error is {e}")
        
    results = []
    car_count = 0
    while car_count < total_cars: 
        
        try:
            page_cars = WebDriverWait(driver, 3).until(EC.presence_of_all_elements_located((By.XPATH, './/div[@class="card-img-wrapper"]')))
            
            for i in range(len(page_cars)): 
                if car_count >= total_cars:
                    break
                
                try:
                    page_cars = WebDriverWait(driver, 3).until(EC.presence_of_all_elements_located((By.XPATH, './/div[@class="card-img-wrapper"]')))
                    driver.execute_script("arguments[0].scrollIntoView();", page_cars[i])
                    time.sleep(1)
                    page_cars[i].click()

                    details = {}  
                    try:
                        det_card = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, './/div[@class="order-lg-2 col-12 col-lg-5 auction-details"]'))) 
                        if det_card:
                            try:
                                title =WebDriverWait(det_card, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="page-title"]'))).text.strip()
                                if title:
                                    details['Title'] = title.replace("\n", " ")
                                else:
                                    details['Title'] = 'na'
                                
                                make = WebDriverWait(det_card, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="page-title"]/h1'))).text.strip()
                                if make:
                                    details['make'] = make
                                else:
                                    details['make'] = 'na'
                                
                                model = WebDriverWait(det_card, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="page-title"]/p'))).text.strip()
                                if model:
                                    details['model'] = model.split(" ")[0]
                                else:
                                    details['model'] = 'na'
                                
                                variant = WebDriverWait(det_card, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="page-title"]/p'))).text.strip()
                                if variant:
                                    details['variant'] = " ".join(variant.split(" ")[1:])
                                else:
                                    details['variant'] = 'na'
                                
                                loc =WebDriverWait(det_card, 5).until(EC.presence_of_element_located((By.XPATH, './/p[@class="auction-location"]'))).text.strip()
                                if loc:
                                    details['Center'] = loc
                                else:
                                    details['Center'] = 'na'
                                    
                            except Exception as e:
                                print("No veh title and location found", e)        
                            try:
                                agent = WebDriverWait(det_card, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="auction-sale"]'))) 
                                if agent:
                                    elements = agent.find_elements(By.TAG_NAME, "p")
                                    if elements:
                                        for ele in elements:
                                            agent_lbl = ele.find_element(By.TAG_NAME, 'strong')
                                            agent_val = ele.text.replace(agent_lbl.text, "").strip()
                                            if agent_lbl and agent_val:
                                                details[agent_lbl.text.strip()] = agent_val
                                            else:
                                                print("No agent data found")
                                    else:
                                        print("No agent elements")
                                    
                                    lane =WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="auction-sale"]/p[3]')))     
                                    if lane:
                                        details['Lane'] = lane.text.strip().split(" ")[1]
                                    else:
                                        details['Lane'] = 'na'                             
                            except Exception as e:
                                print(" No agent card")
                            try:
                                pricing_card = WebDriverWait(det_card, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="price-list"]'))) 
                                if pricing_card:
                                    pri_elements = pricing_card.find_elements(By.TAG_NAME, "p")
                                    if pri_elements:
                                        for pri_ele in pri_elements:
                                            pri_lbl = pri_ele.find_element(By.TAG_NAME, 'strong')
                                            pri_val = pri_ele.text.replace(pri_lbl.text, "").strip()
                                            if pri_lbl and pri_val:
                                                details[pri_lbl.text.strip()] = pri_val
                                            else:
                                                print("No pricing found")
                            except Exception as e:
                                print(" No pricing card")
                            try:
                                footer_card = WebDriverWait(det_card, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="price-footer"]'))) 
                                if footer_card:
                                    fot_ele = footer_card.find_element(By.TAG_NAME, "p")
                                    if fot_ele:
                                        fot_lbl = fot_ele.find_element(By.TAG_NAME, 'strong')
                                        fot_val = fot_ele.text.replace(fot_lbl.text, "").strip()
                                        if fot_lbl and fot_val:
                                            details[fot_lbl.text.strip()] = fot_val
                                        else:
                                            print("No footer pricing found")
                            except Exception as e:
                                print(" No footer pricing card")
                            
                            try:
                                inspec_link = None
                                possible_links = driver.find_elements(
                                    By.XPATH,
                                    './/a[contains(text(), "Download inspection report") or contains(@href, ".pdf")]'
                                )
                                for a in possible_links:
                                    href = a.get_attribute("href")
                                    if href and href.endswith(".pdf"):
                                        inspec_link = href
                                        break

                                if inspec_link:
                                    if inspec_link.startswith("/"):
                                        inspec_link = "https://www.manheim.co.uk" + inspec_link
                                    details["Inspection Report"] = inspec_link
                                else:
                                    details["Inspection Report"] = "na"

                            except Exception as e:
                                print("No inspection report found:", e)
                                details["Inspection Report"] = "na"
                   
                    except Exception as e:
                        print("No main details card")
                    try:
                        non_runner_el = driver.find_element(
                            By.XPATH,
                            './/div[@class="page-title"]//h4//span[contains(text(), "Non Runner")]'
                        )
                        if non_runner_el:
                            details["Non Runner"] = "Non Runner"
                        else:
                            details["Non Runner"] = ""
                    except:
                        details["Non Runner"] = ""

                    try:
                        cat_badge = driver.find_element(
                            By.XPATH,
                            './/div[@class="page-title"]//h4//span[contains(text(), "Cat:")]'
                        ).text.strip()

                        try:
                            accident_detail = driver.find_element(
                                By.XPATH,
                                './/div[@class="page-title"]/p[contains(text(), "Category")]'
                            ).text.strip()
                        except:
                            accident_detail = ""

                        if cat_badge or accident_detail:
                            details["Accident and Damage"] = ", ".join(
                                part for part in [cat_badge, accident_detail] if part
                            )
                        else:
                            details["Accident and Damage"] = ""
                    except:
                        details["Accident and Damage"] = ""

                    try:
                        # Scroll down to bring the inspection tab into view
                        driver.execute_script("window.scrollBy(0, 1000);")
                        time.sleep(1)

                        # Wait for the inspection tab button and click it
                        insp_tab = WebDriverWait(driver, 5).until(
                            EC.element_to_be_clickable((By.XPATH, '//button[@id="spec-tab" and text()="Inspection"]'))
                        )
                        driver.execute_script("arguments[0].scrollIntoView(true);", insp_tab)
                        time.sleep(0.5)
                        insp_tab.click()
                        time.sleep(2)

                        # Wait for the gallery section to appear
                        gallery = WebDriverWait(driver, 5).until(
                            EC.presence_of_element_located((By.XPATH, '//div[contains(@class,"gallerywidget-condition")]'))
                        )

                        # Find all slides
                        slides = gallery.find_elements(By.XPATH, './/div[contains(@class,"slick-slide")]')

                        # damage_images = []
                        damage_details = []

                        for s in slides:
                            try:
                                # Extract image URL
                                img_tag = s.find_element(By.TAG_NAME, 'img')
                                img_url = img_tag.get_attribute("src")

                                # Find overlay div
                                overlay = s.find_element(By.XPATH, './/div[contains(@style,"position:absolute")]')

                                # Extract HTML inside overlay
                                overlay_html = overlay.get_attribute("innerHTML")

                                # Replace <br> with |, remove HTML tags
                                overlay_clean = overlay_html.replace("<br>", " | ")
                                overlay_text = re.sub(r'<.*?>', '', overlay_clean).strip()

                                # damage_images.append(img_url)
                                damage_details.append(overlay_text)

                            except Exception as e:
                                print("⚠️ Skipped one image:", e)
                                continue

                        # details["damage_images"] = damage_images if damage_images else ["na"]
                        details["damage_details"] = damage_details if damage_details else ["na"]

                    except Exception as e:
                        print("❌ Inspection tab or gallery not found:", e)
                        # details["damage_images"] = ["na"]
                        details["damage_details"] = ["na"]

                   
                    
                    try:
                        grade_main = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="gallery-main"]/img'))).get_attribute("src")
                        if grade_main:
                            grade = grade_main.split("/")[-1].split(".")[0]
                            details['Grade'] = grade
                        else:
                            details['Grade'] = 'na'
                    except Exception as e:
                        print("no grade")                            
                    try:
                        img_card =WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="ug-thumbs-strip-inner"]'))) 
                        if img_card:
                            imgs =WebDriverWait(img_card, 5).until(EC.presence_of_all_elements_located((By.TAG_NAME, 'img'))) 
                            if imgs:
                                imgs_lst = [img.get_attribute("src") for img in imgs]
                                details['Images'] = ", ".join(imgs_lst)
                            else:
                                print("No images")
                    except Exception as e:
                        print(f"No image card {e}")
                        
                    try:
                        tabs_card = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/ul[@id="vehicleTab"]')))
                        if tabs_card:
                            try:
                                veh_sum_btn =WebDriverWait(tabs_card, 5).until(EC.presence_of_element_located((By.XPATH, './/button[@id="summary-tab"]'))) 
                                if veh_sum_btn:
                                    driver.execute_script("arguments[0].scrollIntoView();", veh_sum_btn)
                                    veh_sum_btn.click()
                                    try:  
                                        veh_cont_card = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@id="vehicleTabContent"]')))
                                        if veh_cont_card:

                                            try:
                                                add_info = veh_cont_card.find_element(By.XPATH, './/div[@id="vehicleTabContent"]//p').text.strip()
                                                if add_info:
                                                    details['Additional Information'] = add_info
                                                else:
                                                    details['Additional Information'] = 'na'
                                            except Exception as e:
                                                print("No add info")

                                            try:       
                                                veh_sum = WebDriverWait(veh_cont_card, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="auction-summary"]')))
                                                if veh_sum:
                                                    veh_trs = WebDriverWait(veh_sum, 5).until(EC.presence_of_all_elements_located((By.TAG_NAME, 'tr')))
                                                    if veh_trs:
                                                        for veh_tr in veh_trs:
                                                            veh_lbl = veh_tr.find_element(By.TAG_NAME, 'td').text.strip()
                                                            veh_val = veh_tr.find_element(By.TAG_NAME, 'strong').text.strip()
                                                            if veh_lbl and veh_val:
                                                                details[veh_lbl] = veh_val
                                                            else:
                                                                print("No veh details")
                                            except Exception as e:
                                                print("No veh summary info")        
                                        else:
                                            print("No veh summary button")
                                    except Exception as e:
                                        print("No veh summary card found")
                            except Exception as e:
                                print("No veh summary card")

                            try:
                                inspec_btn = WebDriverWait(tabs_card, 5).until(
                                    EC.presence_of_element_located((By.XPATH, './/button[@id="spec-tab"]'))
                                )
                                if inspec_btn:
                                    driver.execute_script("arguments[0].scrollIntoView();", inspec_btn)
                                    inspec_btn.click()
                                    time.sleep(3)

                                    try:
                                        cond_imgs_card = WebDriverWait(driver, 5).until(
                                            EC.presence_of_element_located((By.XPATH, '(.//div[@class="slick-track"])[2]'))
                                        )
                                        if cond_imgs_card:
                                            cond_imgs = WebDriverWait(cond_imgs_card, 5).until(
                                                EC.presence_of_all_elements_located((By.TAG_NAME, 'img'))
                                            )
                                            cond_imgs_lst = [
                                                cond_img.get_attribute("src") for cond_img in cond_imgs if cond_img.get_attribute("src")
                                            ]
                                            details["Damaged_images"] = ", ".join(cond_imgs_lst) if cond_imgs_lst else "na"
                                        else:
                                            print("no cond image card")
                                    except Exception:
                                        print("no cond image card")

                                    try:
                                        int_tyre_card = WebDriverWait(driver, 5).until(
                                            EC.presence_of_element_located((By.XPATH, './/div[@class="row mb+"]'))
                                        )
                                        if int_tyre_card:
                                            general_conditions = []
                                            try:
                                                inspec_rows = int_tyre_card.find_elements(
                                                    By.XPATH, '(.//div[@class="col-md-6"])[1]/table/tbody/tr'
                                                )
                                                for tr in inspec_rows:
                                                    lbl = tr.find_element(By.TAG_NAME, "th").text.strip()
                                                    val = tr.find_element(By.TAG_NAME, "td").text.strip()
                                                    if lbl and val and val != "-":
                                                        general_conditions.append(f"{lbl}: {val}")
                                            except Exception:
                                                pass
                                            tyre_conditions = []
                                            try:
                                                tyre_rows = int_tyre_card.find_elements(
                                                    By.XPATH, '(.//div[@class="col-md-6"])[2]/table/tbody/tr'
                                                )
                                                for tr in tyre_rows:
                                                    tds = [td.text.strip() for td in tr.find_elements(By.TAG_NAME, "td")]
                                                    if len(tds) >= 3:
                                                        tyre_name = tds[0]
                                                        cond1 = tds[1]
                                                        cond2 = tds[2]

                                                        if tyre_name and (cond1 or cond2):

                                                            joined = ", ".join([v for v in [cond1, cond2] if v])
                                                            tyre_conditions.append(f"{tyre_name}: {joined}")
                                            except Exception:
                                                pass


                                            if general_conditions:
                                                details["General Condition"] = ", ".join(general_conditions)
                                            if tyre_conditions:
                                                details["Tyres Condition"] = ", ".join(tyre_conditions)

                                    except Exception as e:
                                        print("No interior and tyre data:", e)

                            except Exception:
                                print("No inspect tab")

                                
                    except Exception as e:
                        print("No buttons found")
                    
                    car_count+=1
                    results.append(details)
                    driver.back()
        
                except Exception as e:
                    print("No cars found")        

            if car_count % 30 == 0: 
                try:
                    next_link = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, './/a[text()="Load More"]')))
                    if next_link:
                        time.sleep(5)
                        next_link.click()
                        
                        WebDriverWait(driver, 3).until(EC.presence_of_all_elements_located((By.XPATH, './/div[@class="card-img-wrapper"]')))
                        print("Hit the next button")
                except Exception as e:
                    print("No next found")
                    break
     
        except Exception as e:
            print("No cars found")
            break
        
    df = pd.DataFrame.from_dict(results)
    df.to_csv("data.csv", index=False)
    driver.quit()

scarpe(833)

🍪 Cookie popup accepted
17 number of cars found
No agent data found
No add info
No veh details
No veh details
No veh details
No veh details
No agent data found
No add info
No veh details
No veh details
No veh details
No veh details
No veh details
No cars found
No cars found
No cars found
No cars found
No cars found
No cars found
No cars found
No cars found
No cars found
No cars found
No cars found
No cars found
No cars found
No cars found
No cars found
No cars found


In [4]:
df = pd.read_csv("data.csv")

df.rename(columns={
    "Sale date:": "Start Date",
    "Sale time:": "Start Time",
    "Lot number:": "Lot",   
    "Cap Clean:": "Cap Clean",
    "Cap Avg.:": "Cap Average",
    "Cap Retail:": "Cap Retail",
    "Registration:": "Reg",
    "Reg year:": "D.O.R",
    "Autotrader Retail Value:": "Autotrader Retail Value",
    "Mileage:": "Mileage",
    "MOT:": "MOT Due",
    "Service History:": "Service History",
    "VAT:": "VAT Status",
    "V5:": "V5",
    "Owners:": "Former Keepers",
    "Transmission:": "Transmission",
    "Fuel:": "Fuel Type",
    "Body:": "Body Type",
    "Doors:": "Doors",
    "Engine size:": "CC",
    "Colour:": "Colour",
    "Service Stamps:": "No of services"
}, inplace=True)

df['Mileage warranted']  = df['Mileage'].apply(lambda x: x.split("\n")[-1] if isinstance(x, str) and "\n" in x else None)
df['Mileage']  = df['Mileage'].apply(lambda x: x.split("\n")[0] if isinstance(x, str) and "\n" in x else None)
df['CC'] = df['CC'].apply(lambda x: f"{int(x.split('cc')[0])/1000:.1f}" if isinstance(x, str) else f"{x / 1000:.1f}")
df['Year'] = df['D.O.R'].apply(lambda x: x.split(" ")[-1] if isinstance(x, str) and " " in x else None)

info_cols = ['Carpet condition', 'Seat condition', 'Upholstery condition']
existing_cols = [col for col in info_cols if col in df.columns]


df['Additional Information'] = df[existing_cols].apply(
    lambda row: {col: row[col] if pd.notnull(row[col]) else 'na' for col in existing_cols},
    axis=1
)


df = df.drop_duplicates(subset='Lot', keep='first')


df.drop(columns=existing_cols + ['Lot'], inplace=True, errors='ignore')

df.to_csv("cca_data.csv", index=False)


KeyError: 'Mileage'

In [ ]:
old_file = "cca_data.csv"
new_file = "cca_data_updated.csv"

df = pd.read_csv(old_file)

def transform_url(old_url):

    if not isinstance(old_url, str) or "dgaww6lqj3" not in old_url:
        return old_url  

    match = re.search(r"motorvehicle/(\d+)/resized/(\d+_[A-Za-z0-9]+)---150-100\.jpg", old_url)
    if match:
        folder_id, image_name = match.groups()
        new_url = f"https://cca-web-public.s3.eu-west-1.amazonaws.com/motorvehicle/{folder_id}/{image_name}.jpg"
        return new_url
    return old_url


def process_images(cell):

    if not isinstance(cell, str):
        return cell
    urls = [u.strip() for u in cell.split(",") if u.strip()]
    new_urls = [transform_url(u) for u in urls]
    return ", ".join(new_urls)


df["Images"] = df["Images"].apply(process_images)


df.to_csv(new_file, index=False)
print(f"✅ Updated file saved as {new_file}")


try:
    os.remove(old_file)
    os.rename(new_file, old_file)
    print("✅ Old file replaced successfully (cca_data.csv updated).")
except Exception as e:
    print(f"⚠️ Error replacing old file: {e}")


🧹 Cleaned column: damage_details
✅ Updated file saved as cca_data_updated.csv
✅ Old file replaced successfully (cca_data.csv updated).


In [ ]:
import ast

file_path = "cca_data.csv"

df = pd.read_csv(file_path)

def clean_damage(cell):
    if pd.isna(cell) or cell.strip() == "['na']":
        return ""
    try:
        items = ast.literal_eval(cell)
        cleaned = []
        for i in items:
            # Remove tabs/newlines and extra spaces
            txt = i.replace("\n", "").replace("\t", "").strip()
            # Compress multiple spaces and add clean separators
            txt = " | ".join([p.strip() for p in txt.split("|") if p.strip()])
            cleaned.append(txt)
        return "; ".join(cleaned)
    except Exception:
        return cell

if "damage_details" in df.columns:
    df["damage_details"] = df["damage_details"].apply(clean_damage)
    df.to_csv(file_path, index=False)
    print("✅ damage_details column cleaned and updated successfully.")
else:
    print("⚠️ 'damage_details' column not found in the CSV.")


✅ damage_details column cleaned and updated successfully.


In [ ]:
import threading
from urllib.parse import urlparse, urljoin
from PIL import Image, ImageDraw, ImageFont


def add_watermark_to_image(image_path, text="Sourced from Central Car Auction"):
    try:
        image = Image.open(image_path).convert("RGBA")
        txt_layer = Image.new("RGBA", image.size, (255, 255, 255, 0))
        draw = ImageDraw.Draw(txt_layer)

        # 🔹 Slightly smaller dynamic font size (half previous)
        font_size = max(20, image.width // 50)  # reduced from //25 → //50
        try:
            font = ImageFont.truetype("arial.ttf", font_size)
        except:
            font = ImageFont.load_default()

        # Calculate text size and position
        margin = int(font_size * 0.6)
        bbox = draw.textbbox((0, 0), text, font=font)
        text_width = bbox[2] - bbox[0]
        text_height = bbox[3] - bbox[1]
        x = image.width - text_width - margin
        y = image.height - text_height - margin

        # Draw semi-transparent background box
        box_padding = int(font_size * 0.4)
        draw.rectangle(
            [x - box_padding, y - box_padding, x + text_width + box_padding, y + text_height + box_padding],
            fill=(0, 0, 0, 180)
        )

        # Draw watermark text
        draw.text((x, y), text, font=font, fill=(255, 255, 255, 240))

        # Merge and save
        watermarked = Image.alpha_composite(image, txt_layer).convert("RGB")
        watermarked.save(image_path)
        print(f"✅ Watermark added to {image_path}")

    except Exception as e:
        print(f"❌ Failed to watermark {image_path}: {e}")



def download_images(data, main_folder="Images", column_name="Images"):
    os.makedirs(main_folder, exist_ok=True)

    for _, row in data.iterrows():
        reg_no = str(row["Reg"]).strip().replace("/", "_").replace("\\", "_")
        image_urls = row.get(column_name)

        if not image_urls or pd.isna(image_urls):
            print(f"⚠️ Skipping {reg_no}: no {column_name.lower()}.")
            continue

        reg_folder = os.path.join(main_folder, reg_no)
        os.makedirs(reg_folder, exist_ok=True)

        urls = [u.strip() for u in image_urls.split(",") if u.strip()]
        for idx, url in enumerate(urls, start=1):
            if "centralcarauctions.com/motorvehicledamageimage" in url:
                url = url.split("key=")[-1].split("&")[0]

            if not url.startswith(("http://", "https://")):
                url = "https://" + url.lstrip("/")

            parsed = urlparse(url)
            if not parsed.scheme or not parsed.netloc:
                print(f"⚠️ Invalid URL skipped: {url}")
                continue

            try:
                response = requests.get(url, stream=True, timeout=15)
                response.raise_for_status()

                ext = os.path.splitext(parsed.path)[1] or ".jpg"
                save_path = os.path.join(reg_folder, f"{idx}{ext}")

                with open(save_path, "wb") as f:
                    for chunk in response.iter_content(1024):
                        f.write(chunk)

                add_watermark_to_image(save_path)
                print(f"✅ Saved: {save_path}")

            except Exception as e:
                print(f"❌ Failed to download {url} for {reg_no}: {e}")



def download_reports(data, main_folder="Inspection Report"):
    os.makedirs(main_folder, exist_ok=True)

    for _, row in data.iterrows():
        reg_no = str(row["Reg"]).strip().replace("/", "_").replace("\\", "_")
        report_url = row.get("Inspection Report")

        if not report_url or pd.isna(report_url):
            print(f"⚠️ Missing Inspection Report for {reg_no}")
            continue

        if not report_url.startswith(("http://", "https://")):
            report_url = "https://" + report_url.lstrip("/")

        reg_folder = os.path.join(main_folder, reg_no)
        os.makedirs(reg_folder, exist_ok=True)
        save_path = os.path.join(reg_folder, f"{reg_no}.pdf")

        try:
            response = requests.get(report_url, stream=True, timeout=15)
            response.raise_for_status()
            with open(save_path, "wb") as f:
                for chunk in response.iter_content(1024):
                    f.write(chunk)
            print(f"✅ Downloaded report: {save_path}")
        except Exception as e:
            print(f"❌ Failed to download {report_url} for {reg_no}: {e}")



def start_funcs():
    df = pd.read_csv("cca_data.csv")
    reports = df[["Reg", "Inspection Report"]]
    reg_img = df[["Reg", "Images"]]
    cond_img = df[["Reg", "Damaged_images"]]

    threads = [
        threading.Thread(target=download_reports, args=(reports,)),
        threading.Thread(target=download_images, args=(reg_img, "Images", "Images")),
        threading.Thread(target=download_images, args=(cond_img, "Damaged_images", "Damaged_images")),
    ]

    for t in threads:
        t.start()
    for t in threads:
        t.join()


if __name__ == "__main__":
    start_funcs()


✅ Downloaded report: Inspection Report\nan\nan.pdf
✅ Downloaded report: Inspection Report\nan\nan.pdf
✅ Downloaded report: Inspection Report\nan\nan.pdf
✅ Watermark added to Damaged_images\nan\1.jpg
✅ Saved: Damaged_images\nan\1.jpg
✅ Downloaded report: Inspection Report\SG72PVV\SG72PVV.pdf
✅ Downloaded report: Inspection Report\nan\nan.pdf
✅ Downloaded report: Inspection Report\nan\nan.pdf
✅ Downloaded report: Inspection Report\SG72WUL\SG72WUL.pdf
✅ Watermark added to Damaged_images\nan\2.jpg
✅ Saved: Damaged_images\nan\2.jpg
✅ Downloaded report: Inspection Report\KM66DGV\KM66DGV.pdf
✅ Watermark added to Images\nan\1.jpg
✅ Saved: Images\nan\1.jpg
✅ Downloaded report: Inspection Report\FP21OSE\FP21OSE.pdf
✅ Downloaded report: Inspection Report\SE70GPZ\SE70GPZ.pdf
✅ Downloaded report: Inspection Report\LK17YYE\LK17YYE.pdf
✅ Watermark added to Damaged_images\nan\3.jpg
✅ Saved: Damaged_images\nan\3.jpg
✅ Downloaded report: Inspection Report\ML17BGZ\ML17BGZ.pdf
✅ Downloaded report: Inspect